## 🧠 What is **KerasTuner**?

**KerasTuner** is a library for **hyperparameter optimization** for Keras models. Instead of manually searching or using `GridSearchCV`, it automates finding the best model by trying many configurations.

---

## 🔍 Why KerasTuner?

- Built specifically for **Keras/TensorFlow**
- Supports **random search**, **Bayesian optimization**, and **Hyperband**
- Better scalability and visualization compared to `GridSearchCV`

---

## 🔧 How it works — Steps:

1. **Define a model-building function**  
   This function includes *searchable hyperparameters* using `hp` (HyperParameters object).

2. **Select a tuner**  
   Choose the strategy: `RandomSearch`, `BayesianOptimization`, `Hyperband`, etc.

3. **Search for best hyperparameters**  
   Call `.search()` to start the process.

4. **Access best model or params**  
   Use `.get_best_models()` or `.get_best_hyperparameters()`.

---

## 📌 What can be asked in exams?

1. **What is KerasTuner?**
2. **Why use it over GridSearchCV?**
3. **What types of tuners are available?**
4. **What is a model-building function?**
5. **How does it handle learning rate, dropout, etc.?**
6. **How do you retrieve the best model and hyperparameters?**

---

In [19]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10

# 1. Load and preprocess CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()

X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# 2. Define the model building function
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(32, 32, 3)))
    model.add(Flatten())
    
    model.add(Dense(units=hp.Choice("units_1", [256, 512]), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Choice("dropout_1", [0.3, 0.4])))

    model.add(Dense(units=hp.Choice("units_2", [128, 256]), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Choice("dropout_2", [0.2, 0.3])))

    model.add(Dense(10, activation='softmax'))

    lr = hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])
    optimizer = Adam(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# 3. Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner_cifar10',
    project_name='cifar10_ann'
)

# 4. Search for the best hyperparameters
tuner.search(X_train, y_train,
             validation_split=0.2,
             epochs=20,
             batch_size=64,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# 5. Get the best model
best_hps = tuner.get_best_hyperparameters(1)[0]
print("\n✅ Best Hyperparameters Found:")
print("Learning Rate:", best_hps.get('learning_rate'))
print("Dropout1:", best_hps.get('dropout_1'))
print("Dropout2:", best_hps.get('dropout_2'))
print("Units1:", best_hps.get('units_1'))
print("Units2:", best_hps.get('units_2'))

# 6. Build and train the best model
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=40,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

# 7. Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\n🎯 Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


Trial 10 Complete [00h 07m 25s]
val_accuracy: 0.4284000098705292

Best val_accuracy So Far: 0.48089998960494995
Total elapsed time: 01h 09m 05s

✅ Best Hyperparameters Found:
Learning Rate: 0.0005
Dropout1: 0.3
Dropout2: 0.2
Units1: 256
Units2: 256
Epoch 1/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2794 - loss: 2.1855 - val_accuracy: 0.3572 - val_loss: 1.8309
Epoch 2/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 9640s 15s/step - accuracy: 0.3694 - loss: 1.7867 - val_accuracy: 0.3045 - val_loss: 2.0214
Epoch 3/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.3913 - loss: 1.7197 - val_accuracy: 0.4141 - val_loss: 1.6648
Epoch 4/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 25s 40ms/step - accuracy: 0.4074 - loss: 1.6692 - val_accuracy: 0.4198 - val_loss: 1.6435
Epoch 5/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.4085 - loss: 1.6496 - val_accuracy: 0.4467 - val_loss: 1.5572
Epoch 6/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.4272 - loss: 1.6188 - val_accurac